### LIFEWISE code

In [6]:
# PART 1: Price vs Quality
# import packages
import pandas as pd
import plotly.express as px
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import linregress
import scipy.stats as stats

# read data
rank = pd.read_csv('input/rank.csv', sep='\t')
reg = pd.read_csv('input/regions.csv', sep='\t')
mrg = pd.merge(rank, reg, on='Country')
# read custom correct data
mrg = pd.read_excel('results/merged_m.xlsx')

# define colors for regions
region_colors = {'Oceania': '#f4a259',
                 'Western_Asia': '#F44336',
                 'Northern_Europe': '#9ed0ab',
                 'Northern_America': '#4b6d8f',
                 'South-eastern_Asia': '#E57373',
                 'Australia': '#aa79d1',
                 'Central_America': '#6d8dc3',
                 'Southern_Asia': '#f9d6b8',
                 'Eastern_Asia': '#EF9A9A',
                 'Southern_Europe': '#bde3c5',
                 'Eastern_Europe': '#d0f0dc',
                 'South_America': '#6d8dc3',
                 'Northern_Africa': '#f9d9a0',
                 'Eastern_Africa': '#f7c36d',
                 'Western_Europe': '#7fcd91',
                 'Southern_Africa': '#e5981d',
                 'Western_Africa': '#edb880',
                 'Central_Asia': '#FFCDD2'}

# Create interactive scatterplot using Plotly Express
custom_order = ['Western_Europe', 'Northern_Europe', 'Eastern_Europe', 'Southern_Europe',
                'Western_Asia', 'Central_Asia', 'Eastern_Asia', 'Southern_Asia',
                'Northern_Africa', 'Western_Africa', 'Eastern_Africa', 'Southern_Africa',
                'South_America', 'Central_America', 'South-eastern_Asia', 'Australia',
                'Northern_America']

fig = px.scatter(mrg, x='Quality_of_Life_Index', y='Cost_of_Living_Index', color='Region',
                 color_discrete_map=region_colors, symbol='Region', hover_name='Country',
                 width=800, height=600, category_orders={'Region': ['Australia', 'Central_America', 'Northern_America', 'South_America',
                                             'Eastern_Africa', 'Northern_Africa', 'Southern_Africa', 'Western_Africa',
                                             'Eastern_Asia', 'Southern_Asia', 'South-eastern_Asia', 'Western_Asia','Central_Asia',
                                             'Eastern_Europe', 'Northern_Europe', 'Southern_Europe', 'Western_Europe']})

# Modify the legend and layout
fig.update_traces(marker=dict(size=10, line=dict(width=0.5, color='Gray')),
                  selector=dict(mode='markers'))
fig.update_layout(legend=dict(title='', orientation='v', yanchor='top', y=0.99, xanchor='left', x=1.02),
                  font=dict(family='Arial', size=14),
                  margin=dict(l=40, r=40, t=20, b=20), 
                  xaxis_title='Quality of Life Index', yaxis_title='Cost of Living Index',
                  width=900, height=600, template='plotly_white')

import plotly.io as pio
pio.write_html(fig, file='results/Cost_vs_quality.html', auto_open=True)

# Add to data information about Cost to quality ratio and quantiles
mrg['Cost_vs_quality_ratio'] = mrg['Quality_of_Life_Index']/mrg['Cost_of_Living_Index']
mrg['quantiles'] = pd.qcut(mrg['Cost_vs_quality_ratio'], q=4, labels=['bad_high','bad_low','good_low', 'good_high'])
mrg.to_excel('results/LQI_quantiles.xlsx', index=False)

# Visualize quantiles information
quantiles_colors = {'good_high': '#759116',
                 'good_low': '#056517',
                 'bad_low': '#f57a9b',
                 'bad_high': '#de1a24'}

fig = px.scatter(mrg, x='Quality_of_Life_Index', y='Cost_of_Living_Index', color='quantiles',
                 symbol='quantiles', hover_name='Country', color_discrete_map=quantiles_colors,
                 category_orders={'quantiles': ['good_high', 'good_low','bad_low','bad_high']},
                 width=800, height=600)

# Modify the legend and layout
fig.update_traces(marker=dict(size=10, line=dict(width=0.5, color='Gray')),
                  selector=dict(mode='markers'))
fig.update_layout(legend=dict(title='', orientation='v', yanchor='top', y=0.99, xanchor='left', x=1.02),
                  font=dict(family='Arial', size=14),
                  margin=dict(l=40, r=40, t=20, b=20), 
                  xaxis_title='Quality of Life Index', yaxis_title='Cost of Living Index',
                  width=900, height=600, template='plotly_white')

pio.write_html(fig, file='results/Quantiles.html', auto_open=True)

# Choose top countries with hight life quality index and good Cost to quality ratio 
mrg1 = mrg[(mrg['Quality_of_Life_Index'] >= 160) & 
           ((mrg['quantiles'] == 'good_high') | (mrg['quantiles'] == 'good_low'))]

mrg1.to_excel('results/top_counties.xlsx')


/var/folders/fr/j_4kmkcj6zs6nglntg64jfgr0000gn/T/ipykernel_82673/2861344883.py:65: UserWarning:

Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.1' currently installed).

/var/folders/fr/j_4kmkcj6zs6nglntg64jfgr0000gn/T/ipykernel_82673/2861344883.py:93: UserWarning:

Pandas requires version '1.4.3' or newer of 'xlsxwriter' (version '1.3.1' currently installed).



In [ ]:
# Part 2. Predictions
# import packages
import pandas as pd
import plotly.express as px
import numpy as np
from sklearn import datasets, linear_model
import matplotlib.pyplot as plt
from scipy.stats import linregress

mrg = pd.read_excel('results/merged_m.xlsx')

# build a model
# extract the columns you need
X = mrg['Cost_of_Living_Index'].values.reshape(-1,1)
y = mrg['Quality_of_Life_Index'].values.reshape(-1,1)
# create a linear regression model
regr = linear_model.LinearRegression()
regr.fit(X, y)

# Print the formula and coefficients
from scipy import stats

X1 = mrg['Cost_of_Living_Index']
y1 = mrg['Quality_of_Life_Index']
slope, intercept, r_value, p_value, std_err = stats.linregress(X1, y1)

print('Formula is: y = ', slope.round(1), 'x + ', intercept.round(1))
print('Where y is the Quality of Life Index and x is the Cost of Living Index','\n')
print("r-squared:", (r_value**2*100).round(1))
print("p_value:", p_value)
print("p_value:", std_err)

cost = 2400
# in EUR per month
NY_cost_of_living = 4900
lcost_index = cost/NY_cost_of_living*100
lqual_index = regr.predict(lcost_index)[0][0]

print('The predicted life quality index for '
      +str(cost)+' EUR life cost is', str(lqual_index.round(1)))
bins = np.percentile(mrg['Quality_of_Life_Index'], np.arange(0, 101, 10))
new_value_bin = np.digitize(lqual_index, bins) 

d = {
  10: "top 10% best",
  9: "top 20% best",
  8: "top 30% best",
  7: "40% best",
  6: "50% best",
  5: "50% worst",
  4: "40% worst",
  3: "top 30% worst",
  2: "top 20% worst",
  1: "top 10% worst"
}

print(f'This index refers to the {d[new_value_bin]} countries in terms of standard of living','\n')
print('REMEMBER! r2=0.51 for this model. That means the model has quite low predictive power.')